In [20]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jul_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jul_dict_JSON = json.load(json_file)

In [21]:
# week_day_Jul_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]
week_day_Jul_list = [2, 9, 16, 23, 30, 31]

In [22]:
link_day_minute_Jul_list = []
for link_idx in range(24):
    for day in week_day_Jul_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jul_list.append(link_day_minute_Jul_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [23]:
len(link_day_minute_Jul_list)

17280

In [24]:
x = np.matrix(link_day_minute_Jul_list)
x = np.matrix.reshape(x, 24, 720)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [25]:
np.size(x, 0), np.size(x, 1)

(24, 720)

In [26]:
x[:,:2]

matrix([[  2136.06674665,   2008.03265958],
        [  2775.0543623 ,   5468.17490943],
        [  3033.91173729,   3047.60914358],
        [  5158.83944197,   5394.98263971],
        [  3182.06559307,   3444.44727344],
        [  9103.90874818,   8867.68405924],
        [  2678.65145695,   2640.19073646],
        [  2748.06623017,   3296.74133768],
        [  6257.27450378,   8436.71914537],
        [ 10663.45670255,  10991.55537808],
        [  7545.06461598,   8896.28419843],
        [  8125.21554502,   8728.30233267],
        [  7507.71498527,   6131.11623836],
        [  3010.41299015,   1051.25994621],
        [  6769.72409266,   3680.54976191],
        [  1517.99174628,   1984.67118849],
        [  2701.27571664,   3095.10067561],
        [  1788.12023883,   2238.52827238],
        [ 12473.87484785,  13375.45259996],
        [  2315.29324646,   2472.40915216],
        [ 10656.4209806 ,  11190.66546003],
        [  4000.83198032,   4325.07903563],
        [  2582.1359908 ,   3037

In [27]:
np.size(A,0), np.size(A,1)

(24, 140)

In [28]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [2e-02, 3e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.01s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -6.33842012e+06  0.00000000e+00  5.83e+03 3.93e+02  1.00e+06     0s
   1  -8.52649489e+03 -2.81102190e+06  1.04e+03 1.78e+02  2.36e+05     0s
   2   1.24173996e+06 -2.74829849e+06  2.16e+00 1.45e-01  7.17e+04     0s
   3  -2.00092437e+05 -7.64632631e+05  2.16e-06 1.45e-07  1.0

In [29]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jul_weekday_AM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [30]:
lam_list

[6.539541572964988e-10,
 3055.9841262455293,
 7.178027065695695e-10,
 2.67567727571332e-06,
 1070.2325506790412,
 4.405963475041796e-06,
 2.0649264673978614e-07,
 2635.282555987744,
 1061.712555719129,
 3360.988427122535,
 2.6380315398567833e-06,
 410.81091761048117,
 4.333330366868872e-06,
 4.40006634976715e-06,
 254.80849737112038,
 208.03869772884337,
 8.694466383927417e-07,
 2.6812500571035333e-06,
 1948.7892187350276,
 11535.309000003037,
 4.380023113322029e-06,
 2.970935574005129e-05,
 4065.0660150125623,
 2.9804513620665125e-06,
 1680.1580198397733,
 1442.9838133531568,
 6832.664165413636,
 2736.9499295628266,
 6413.770703933681,
 6164.275943778972,
 2.7951029263129697e-06,
 1826.9911606268374,
 5617.174982634314,
 0.00010356046409532721,
 3.4521996368826885e-05,
 1.893884612075465e-05,
 1.781743568061815e-05,
 9062.590861777968,
 9.236202258077831e-07,
 2614.0473449505475,
 4069.3959499350253,
 1952.1986646872479,
 1042.2940296963154,
 1077.3895699992509,
 3.631458475017099e-06